Let's first grab the necessary libraries
----

In [1]:
import tensorflow as tf
import numpy as np

Now, let's get the basic structure of the RNN in Tensorflow
---

In [2]:
# Basic structure of the RNN
n_inputs  = 3
n_neurons = 5

# Input Matrix Layers
X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

# Output connection weights
Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32))

# Input connection weights
Wy = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32))

# Bias term
b  = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))

# Output Matrix Layers
Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
Y1 = tf.tanh(tf.matmul(X0, Wy) + tf.matmul(X1, Wx) + b)

# Create a Global Variable creation method
init = tf.global_variables_initializer()

Great! Now we've got a Two-Layer Feedforward Neural Network!
---
Let's give it minibatch capabilities.

In [3]:
# Mini-batch:
X0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]])
X1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]])

# Run the sessions
with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})
    
print(Y0_val) # output at t = 0
print(Y1_val) # output at t = 1

[[ 0.48568213  0.99465543  0.9956983   0.1421937  -0.73680246]
 [-0.054739    1.          0.9999998   0.99980026  0.07819676]
 [-0.56488687  1.          1.          1.          0.8005569 ]
 [-0.9999931   0.998329   -0.99933213  1.          0.9923835 ]]
[[-0.99911094  1.          1.          1.          0.9999999 ]
 [-0.9977052   1.         -0.989354   -1.          1.        ]
 [-0.99999803  1.          0.95064473 -1.          1.        ]
 [ 0.99419284  1.         -0.99481493 -1.          1.        ]]


Static Unrolling Through Time of RNN
---
Instead of the basic model, let's push a tensor to the model pre-packed with mini-batch sequences.

In [3]:
# Basic structure of the RNN
n_inputs  = 3
n_neurons = 5
n_steps   = 2

# Prepack a tensor with n_steps worth of placeholders from n_inputs
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
X_seqs = tf.unstack(tf.transpose(X, perm=[1, 0, 2]))

# Setup the Tensorflow object for the RNN
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, X_seqs, dtype=tf.float32)

# Formulate the output
outputs = tf.transpose(tf.stack(output_seqs), perm=[1, 0, 2])

init = tf.global_variables_initializer()

Now we've got a Static RNN Module ready to receive data
---
Let's pack the tensor, and start the job just like before.

In [4]:
# Place the two steps into an np.array
X_batch = np.array([
    [[0, 1, 2], [9, 8, 7]],
    [[3, 4, 5], [0, 0, 0]],
    [[6, 7, 8], [6, 5, 4]],
    [[9, 0, 1], [3, 2, 1]],
])

# Run the model, feeding the X_batch data into X
with tf.Session() as sess:
    init.run()
    outputs_val = outputs.eval(feed_dict={X: X_batch})
    
# Output the final weights
print(outputs_val)

# Page 510

[[[-3.5058382e-01  1.5799947e-01 -9.2109334e-01 -8.1341058e-01
    2.6493451e-01]
  [ 9.5667464e-01 -9.9999887e-01 -9.9850333e-01 -9.9822176e-01
    9.9996877e-01]]

 [[ 1.8639387e-01 -9.6400917e-01 -9.9784625e-01 -9.8885560e-01
    9.4688809e-01]
  [ 3.9655499e-02 -7.8954637e-01  5.3778267e-01  1.5927261e-01
    9.0873235e-01]]

 [[ 6.3113898e-01 -9.9951184e-01 -9.9994332e-01 -9.9938983e-01
    9.9744189e-01]
  [ 8.6245078e-01 -9.9997514e-01 -8.8232809e-01 -9.1973686e-01
    9.9968272e-01]]

 [[-2.4933994e-01 -9.9998391e-01  9.9914992e-01 -4.6435291e-01
    9.9999189e-01]
  [ 9.6336168e-01 -9.7827965e-01 -6.0130131e-01 -1.7094611e-04
    8.0779636e-01]]]
